# Course 4 - Project - Part 1: Feature extraction

<a name="top-1"></a>
This notebook is concerned with *Part 1: Feature extraction*.

**Contents:**
* [Step 1: Take a first look at the dataset](#step-1.1)
* [Step 2: Set up a pretrained model](#step-1.2)
* [Step 3: Extract features](#step-1.3)

## Step 1: Take a first look at the dataset<a name="step-1.1"></a> ([top](#top-1))
---

We begin with some imports.

In [1]:
# Standard library.
import pathlib

**Note:** We assume that the Swissroads dataset has been downloaded and extracted into a directory named _data/_.

In [2]:
base_path = pathlib.Path.cwd() / 'data' / 'swissroads'
assert base_path.is_dir()

We see that the dataset is rather small (469 images) and has already been divided into 3 subsets for training, validation and test.

In [3]:
for kind in ['train', 'valid', 'test']:
    path = base_path / kind
    num = sum(1 for _ in path.glob('**/*.png'))
    print(f'{kind}: {num} images')

train: 280 images
valid: 139 images
test: 50 images


## Step 2: Set up a pretrained model<a name="step-1.2"></a> ([top](#top-1))
---

We begin with some imports.

In [4]:
# 3rd party.
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
tf.__version__

'1.12.0'

We decide to use the MobileNet v2 CNN model from TensorFlow Hub.

In [6]:
# MobileNet V2.
# (An updated implementation version exists but seems to require TF 1.15 or TF 2.)
MOBILENET_V2_VERSION = 3  # implementation version
MOBILENET_V2_URL = f'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/{MOBILENET_V2_VERSION}'

We download and setup the pretrained model.

In [7]:
# Create graph.
img_graph = tf.Graph()

with img_graph.as_default():
    # Download module.
    module_url = MOBILENET_V2_URL
    module = hub.Module(module_url)
    
    # Get the expected size.
    height, width = hub.get_expected_image_size(module)
    
    # Create an input placeholder.
    # ? [samples] x height [pixels] x width [pixels] x 3 [color channels]
    input_imgs = tf.placeholder(dtype=tf.float32, shape=[None, height, width, 3])
    
    # Get a node with the features.
    imgs_features = module(input_imgs)
    
    # Collect the initializers.
    init_op = tf.group([
        tf.global_variables_initializer(), tf.tables_initializer()
    ])
    
img_graph.finalize()  # Make the graph "read-only".

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Step 3: Extract features<a name="step-1.3"></a> ([top](#top-1))
---

We begin with some imports.

In [8]:
# Standard library.
import os
import pathlib
import typing as T

# 3rd party.
import numpy as np

# Project.
import datasetutils

As per the documentation of MobileNet v2, we need to resize images and scale color channels:

> The input images are expected to have color values in the range [0,1], following the common image input conventions. For this module, the size of the input images is fixed to height x width = 224 x 224 pixels.

**_Note:_** We need to load and process images while keeping track of their labels. It may be possible to "hijack" an `ImageDataGenerator` by disabling all data augmentations and reading the exact number of images. Here, we decide to do it by hand. The corresponding code is in **datasetutils.py**.

We process the training, validation and test datasets. Since the datasets are rather small, we decide to use bicubic interpolation when resizing the images and to save both images and extracted features in the same NPZ file (this is most likely less efficient than PNG compression).

In [9]:
separator = ''.center(80, '-')

# Create a session.
with tf.Session(graph=img_graph) as sess:
    # Initialize the session.
    sess.run(init_op)
    
    # Extract the features.
    for kind in ['train', 'valid', 'test']:
        print(separator)
        print(f'Dataset: {kind}')
        
        # Load the dataset.
        path = base_path / kind
        print(f'Loading dataset ({path})...')
        dataset = datasetutils.load_dataset(path)
        
        # Extract the features and add them to the dataset.
        print('Extracting features...')
        features = sess.run(imgs_features, feed_dict={input_imgs: dataset['data']})
        print(f'Features: shape={features.shape}, dtype={features.dtype}')
        dataset['features'] = features
        
        # Save the dataset.
        ouput_path = pathlib.Path.cwd() / 'data' / f'swissroads-features-{kind}.npz'
        print(f'Saving ({ouput_path})...')
        np.savez(ouput_path, **dataset)

--------------------------------------------------------------------------------
Dataset: train
Loading dataset (/Users/taariet1/cont-edu/Adsml/git/course-04-project/data/swissroads/train)...
Extracting features...
Features: shape=(280, 1280), dtype=float32
Saving (/Users/taariet1/cont-edu/Adsml/git/course-04-project/data/swissroads-features-train.npz)...
--------------------------------------------------------------------------------
Dataset: valid
Loading dataset (/Users/taariet1/cont-edu/Adsml/git/course-04-project/data/swissroads/valid)...
Extracting features...
Features: shape=(139, 1280), dtype=float32
Saving (/Users/taariet1/cont-edu/Adsml/git/course-04-project/data/swissroads-features-valid.npz)...
--------------------------------------------------------------------------------
Dataset: test
Loading dataset (/Users/taariet1/cont-edu/Adsml/git/course-04-project/data/swissroads/test)...
Extracting features...
Features: shape=(50, 1280), dtype=float32
Saving (/Users/taariet1/cont-